In [8]:
import multiprocessing
import os

import cv2
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from albumentations import *
from albumentations.pytorch import ToTensorV2

## Config

In [9]:
# config
class Config():
    seed = 42
    
    # 경로
#     data_dir = './face_input/train' #
    data_dir = './input/data/eval'
    # 모델 경로
    save_model_dir = './kfoldEnsemble_exp6'
    # Output 경로
    output_dir = './kfoldEnsemble_output6'
    resize = [224, 224]
    
    # 추론 설정
    batch_size = 1000
    n_splits = 5 # k - fold

config = Config()

 # -- settings
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## test dataset

In [10]:
class TestDataset(Dataset):
    def __init__(self, img_paths, resize, mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
        self.img_paths = img_paths
        self.transform = Compose([
            CenterCrop(height=480, width=320),
            Resize(resize[0], resize[1], p=1.0),
            Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

    def __getitem__(self, index):
        # image = Image.open(self.img_paths[index])
        image_path = self.img_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transform:
            image = self.transform(image=np.array(image))['image']
        return image

    def __len__(self):
        return len(self.img_paths)

In [11]:
img_root = os.path.join(config.data_dir, 'images')
info_path = os.path.join(config.data_dir, 'info.csv')
info = pd.read_csv(info_path)
img_paths = [os.path.join(img_root, img_id) for img_id in info.ImageID]

dataset = TestDataset(img_paths, config.resize)
loader = DataLoader(
    dataset,
    batch_size=config.batch_size,
    num_workers=multiprocessing.cpu_count() // 2,
    shuffle=False,
    pin_memory=use_cuda,
    drop_last=False,
)

## Model

In [12]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
class ResnextMultiheadModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.base_model = models.resnext50_32x4d(pretrained=True)
        self.base_model.fc = Identity()

        self.fc_mask_classifier = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=3, bias=True)
        )
        self.fc_age_classifier = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=1000, bias=True),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=3, bias=True)
        )
        self.fc_gender_classifier = nn.Sequential(
            nn.Linear(in_features=2048, out_features=1000, bias=True),
            nn.ELU(True),
            nn.Dropout(0.5, inplace=True),
            nn.Linear(in_features=1000, out_features=2, bias=True)
        )
    def forward(self, x):
        x = self.base_model(x)
        mask = self.fc_mask_classifier(x)
        age = self.fc_age_classifier(x)
        gender = self.fc_gender_classifier(x)
        return mask, age, gender

In [13]:
def load_model(save_model_dir, num, device):
    model = ResnextMultiheadModel()

    model_path = os.path.join(save_model_dir, f'best_{num}.pth')
    model.load_state_dict(torch.load(model_path, map_location=device))

    return model

## Inference

In [14]:
oof_pred = None
for i in range(config.n_splits):
    print(f"Calculating inference results..[{i}/{config.n_splits}]")
    model = load_model(config.save_model_dir, i, device).to(device)
    model.eval()
    mask_predictions = []
    age_predictions = []
    gender_predictions = []
    with torch.no_grad():    
        for images in loader:
            images = images.to(device)
            mask_outs, age_outs, gender_outs = model(images)
            mask_predictions.extend(mask_outs.cpu().numpy())
            age_predictions.extend(age_outs.cpu().numpy())
            gender_predictions.extend(gender_outs.cpu().numpy())
    fold_mask_pred = np.array(mask_predictions)
    fold_age_pred = np.array(age_predictions)
    fold_gender_pred = np.array(gender_predictions)
    
    if oof_pred is None:
        oof_mask_pred = fold_mask_pred / config.n_splits
        oof_age_pred = fold_age_pred / config.n_splits
        oof_gender_pred = fold_gender_pred / config.n_splits
    else:
        oof_mask_pred += fold_mask_pred / config.n_splits
        oof_age_pred += fold_age_pred / config.n_splits
        oof_gender_pred += fold_gender_pred / config.n_splits

mask_preds = np.argmax(oof_mask_pred, axis=1)
age_preds = np.argmax(oof_age_pred, axis=1)
gender_preds = np.argmax(oof_gender_pred, axis=1)

preds = mask_preds * 6 + gender_preds * 3 + age_preds

info['ans'] = preds
os.makedirs(config.output_dir)
save_path = os.path.join(config.output_dir, f'output.csv')
info.to_csv(save_path, index=False)
print(f"Inference Done! Inference result saved at {save_path}")

Calculating inference results..[0/5]
Calculating inference results..[1/5]
Calculating inference results..[2/5]
Calculating inference results..[3/5]
Calculating inference results..[4/5]
Inference Done! Inference result saved at ./kfoldEnsemble_output6/output.csv
